In [1]:
%%capture
!pip install -q datasets evaluate #requirements.txt

In [2]:
import torch
import pandas as pd
import numpy as np
from evaluate import load
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline

# Loading Data

## Utils

In [3]:
def prepare_dataset(file_path):
    """
    Prepares the dataset by splitting it into training and testing sets, and converts the 'text' and 'text_en' columns
    into a format that can be used for fine-tuning, with the same split for both columns.

    Arguments:
    file_path : str : The file path to the dataset in TSV format.

    Returns:
    datasets.DatasetDict : A dictionary containing the train and test datasets formatted for fine-tuning,
                            with both 'text' and 'text_en' columns.
    """
    # Load the dataset from the given TSV file path using the datasets library
    raw_dataset = load_dataset('csv', data_files=file_path, delimiter='\t')

    # Convert the dataset to a pandas DataFrame for easier manipulation
    raw_dataframe = raw_dataset['train'].to_pandas()

    # Extract the text and label columns as lists
    X = raw_dataframe[['text', 'text_en']]  # Use 'text' and 'text_en' columns
    y = raw_dataframe['label'].tolist()  # 'label' is assumed to be the column for target labels

    # Split the data into training and testing sets (90% training, 10% testing)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

    # Convert the training and testing data into Hugging Face dataset format (DatasetDict)
    dataset = DatasetDict({
        'train': Dataset.from_dict({
            'text': X_train['text'].tolist(),
            'text_en': X_train['text_en'].tolist(),
            'label': y_train
        }),
        'test': Dataset.from_dict({
            'text': X_test['text'].tolist(),
            'text_en': X_test['text_en'].tolist(),
            'label': y_test
        })
    })

    # Return the prepared dataset
    return dataset


In [4]:
def select_dataset_text_column(dataset, text_column):
    """
    Converts the dataset to use a specific text column ('text' or 'text_en') and returns the train and test splits
    with the specified column and 'label' column.

    Arguments:
    dataset : datasets.DatasetDict : The prepared dataset in Hugging Face format (train and test sets).
    text_column : str : The column to use for text data ('text' or 'text_en').

    Returns:
    datasets.DatasetDict : A dictionary containing the train and test datasets with the specified text column.
    """
    # Convert the training and testing data into Hugging Face dataset format (DatasetDict)
    dataset_with_text_column = DatasetDict({
        'train': Dataset.from_dict({
            'text': dataset['train'][text_column],  # Select the specified text column
            'label': dataset['train']['label']  # Keep the 'label' column
        }),
        'test': Dataset.from_dict({
            'text': dataset['test'][text_column],  # Select the specified text column
            'label': dataset['test']['label']  # Keep the 'label' column
        })
    })

    # Return the modified dataset with the selected text column
    return dataset_with_text_column

In [5]:
# Tokenizes the input data using the provided tokenizer.
def tokenize_function(data, tokenizer):
    """
    Arguments:
    data : dict : A dictionary containing the text data.
    tokenizer : transformers.PreTrainedTokenizer : The tokenizer to use for tokenizing the text.

    Returns:
    dict : Tokenized data with 'input_ids' and 'attention_mask'.
    """
    return tokenizer(data['text'], padding="max_length", truncation=True)


## Main

In [6]:
# File path for the dataset
file_path = "orientation-it-train.tsv"

In [7]:
dataset = prepare_dataset(file_path)

Generating train split: 0 examples [00:00, ? examples/s]

# Fine-tuning

## Loading Model

In [8]:
# fine-tuning model
model_name = "FacebookAI/xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenizing data for training

In [9]:
# For power identification -> text is used
dataset_text = select_dataset_text_column(dataset, 'text')

# Tokenize the data
tokenized_datasets = dataset_text.map(lambda x: tokenize_function(x, tokenizer), batched=True)

Map:   0%|          | 0/3030 [00:00<?, ? examples/s]

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

## Fine-tuning Utils

In [14]:
# Returns the appropriate training arguments based on the dataset name.
def get_training_arguments():
    """
    Arguments:
    dataset_name : str : The name of the dataset (or any identifier to switch between datasets)

    Returns:
    transformers.TrainingArguments : Training arguments configured based on the dataset.
    """
    return TrainingArguments(
        output_dir='./results_power',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        logging_dir="./results_power/logs",
        save_total_limit=2,
        fp16=True,
        report_to=[]
    )


In [11]:
# Computes accuracy as the metric.
def compute_metrics(eval_pred):
    """
    Arguments:
    eval_pred : tuple : Contains logits and true labels for evaluation.

    Returns:
    dict : A dictionary containing the accuracy of the predictions.
    """
    metric = load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
# Fine-tunes a sequence classification model on the given dataset.
def fine_tune_model(dataset, model_name, tokenizer):
    """
    Arguments:
    dataset : datasets.DatasetDict : A dictionary containing the train and test datasets.
    model_name : str : The name of the pre-trained model to use.
    tokenizer : transformers.PreTrainedTokenizer : The tokenizer for tokenizing the text.
    dataset_name : str : The name of the dataset (e.g., 'orientation', 'power').

    Returns:
    transformers.Trainer : The trained model.
    """
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Get dataset-specific training arguments
    training_args = get_training_arguments()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,  # This function is defined above
        #callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Add early stopping
    )

    trainer.train()
    return trainer


## Main

In [15]:
# Clear CUDA cache (ensure no memory leakage)
torch.cuda.empty_cache()

# Fine-tune the model on 'power' dataset
trainer = fine_tune_model(tokenized_datasets, model_name, tokenizer)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-4a392649ca68>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.663829,0.617211
2,No log,0.634834,0.623145
3,0.659100,0.580891,0.697329


In [16]:
# Get the predictions and labels
predictions = trainer.predict(tokenized_datasets['test'])
predicted_labels = predictions.predictions.argmax(axis=-1)  # Get the predicted class labels
true_labels = predictions.label_ids

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(classification_report(true_labels, predicted_labels))


Accuracy: 69.73%
              precision    recall  f1-score   support

           0       0.74      0.33      0.45       129
           1       0.69      0.93      0.79       208

    accuracy                           0.70       337
   macro avg       0.71      0.63      0.62       337
weighted avg       0.71      0.70      0.66       337



# Inference

## Loading Model

In [17]:
torch.cuda.empty_cache()


In [18]:
# LLM model for inference
model_name = "facebook/bart-large-mnli"
pipe = pipeline("zero-shot-classification", model=model_name, device="cuda")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda


## Utils

In [19]:
metric = load("accuracy")

In [20]:
# Perform zero-shot inference on a dataset using a pre-defined pipeline.
def perform_inference(dataset, candidate_labels):
    """
    Arguments:
    dataset : DatasetDict : The dataset object containing a 'test' split with a 'text' field.
    candidate_labels : list : A list of candidate labels for zero-shot classification.

    Returns:
    list : A list of indices corresponding to the highest-confidence label for each prediction.
    """
    # Perform zero-shot classification on the test dataset's 'text' field
    zeroshot_preds = pipe(
        dataset['test']['text'],  # Input text data from the test split
        batch_size=16,           # Process the data in batches of size 16 for efficiency
        candidate_labels=candidate_labels  # Specify the candidate labels for classification
    )

    # Print the raw predictions for debugging or inspection
    print(zeroshot_preds)

    # Convert predictions to indices of the highest-confidence label for each instance
    return [candidate_labels.index(pred["labels"][0]) for pred in zeroshot_preds]


## On English Text

In [21]:
dataset_text_en = select_dataset_text_column(dataset, 'text_en')

In [22]:
torch.cuda.empty_cache()

# Inference on "text_en" (English)
inference_text_en = perform_inference(dataset_text_en, [0, 1])
#metric = load("accuracy") # defined above already
accuracy_text_en = metric.compute(predictions=inference_text_en, references=dataset_text_en["test"]["label"])

print("Accuracy on English text:", accuracy_text_en)

[{'sequence': "I'm asking to talk. <p> Matteoli (<PARTY>). Mr President, on behalf of the Forza Italia Group, I wish to express my condolences to the families of the eight victims of the Livorno flood and solidarity with the city. I am particularly attached to Livorno, because he elected me for ten years city councilor and for five years provincial councilor. When a family, the typical Italian family, composed of its father, mother, grandparents and grandchildren, lives in symbiosis and suddenly a wave of water and mud overwhelms it and leaves only a two-year-old girl alive and grandmother, or, in a couple, the wife dies leaving her husband, it is clear that we have to ask ourselves why. Fortunately, the General Insurances made a gesture, which I appreciated and which I believe all colleagues also appreciate: they will think of the future of the child, since both the grandfather and the father had an insurance agency of the Generals. Is there the imponderable element? I don't know. But

In [23]:
# Compute classification report
classification_report_text_en = classification_report(dataset_text_en["test"]["label"], inference_text_en)
print("Classification Report on English text:")
print(classification_report_text_en)

Classification Report on English text:
              precision    recall  f1-score   support

           0       0.52      0.22      0.31       129
           1       0.64      0.88      0.74       208

    accuracy                           0.62       337
   macro avg       0.58      0.55      0.52       337
weighted avg       0.60      0.62      0.57       337



## On Original Text

In [24]:
dataset_text = select_dataset_text_column(dataset, 'text')

In [25]:
# Inference on "text" (Original Language)
inference_text = perform_inference(dataset_text, [0, 1])
#metric = load("accuracy") # defined above already
accuracy_text = metric.compute(predictions=inference_text, references=dataset_text["test"]["label"])

print("Accuracy on Original text:", accuracy_text)

[{'sequence': "Domando di parlare. <p> MATTEOLI (<PARTY>). Signor Presidente, a nome del Gruppo di Forza Italia esprimo cordoglio per le famiglie delle otto vittime dell'alluvione di Livorno e solidarietà alla città. Sono particolarmente legato a Livorno, perché mi ha eletto per dieci anni consigliere comunale e per cinque anni consigliere provinciale. Quando una famiglia, la tipica famiglia italiana, composta dal padre, dalla madre, dai nonni e i nipotini, vive in simbiosi e, all'improvviso, un'onda di acqua e di fango la travolge e lascia in vita solo una bambina di due anni e la nonna, oppure, in una coppia, la moglie muore lasciando il marito, è chiaro che dobbiamo interrogarci sul perché. Per fortuna, le Assicurazioni Generali hanno fatto un gesto, che io ho apprezzato e che credo apprezzino anche tutti i colleghi: penseranno loro al futuro della bambina, dal momento che sia il nonno che il padre avevano una agenzia di assicurazioni delle Generali. Vi è l'elemento imponderabile? I

In [26]:
# Compute classification report
classification_report_text = classification_report(dataset_text["test"]["label"], inference_text)
print("Classification Report on Original text:")
print(classification_report_text)

Classification Report on Original text:
              precision    recall  f1-score   support

           0       0.41      0.22      0.29       129
           1       0.62      0.80      0.70       208

    accuracy                           0.58       337
   macro avg       0.52      0.51      0.50       337
weighted avg       0.54      0.58      0.54       337

